In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
class raw_eyelink():
    
    def __init__(self, asc_fname):
        self.asc_fname = Path(asc_fname)
        
    def parse(self):
        with self.asc_fname.open() as file:
            samples = [] 
            events = {}

            is_recording_block = False
            for line in file:
                if line.isspace():
                    continue
                line = line.split()
                if line[0] == 'START':
                    is_recording_block = True
                if is_recording_block:
                    if line[0].isdigit():  # Sample lines start with a number.
                        samples.append(line)
                    elif line[0].isupper():  # Event strings are all-caps
                        if line[0] not in events.keys():
                            events[f'{line[0]}'] = [line[1:]]
                        else: # append line to existing key
                            events[f'{line[0]}'].append(line[1:])                
                if line[0] == 'END':
                    is_recording_block = False
            return samples, events
    
    def _event_to_dataframe(self, events, event, column_names, empty_dict):
        empty_dict[event[1:]] = pd.DataFrame(events[event],
                                                    columns=column_names)
        empty_dict[event[1:]]['type'] = event[1:].lower()
        empty_dict[event[1:]].replace('.', np.NaN, inplace=True)
        
    def to_dataframe(self):
        events_dataframes = {}
        timestamp_cols = ['eye','time_stamp','end_timestamp','duration',]
        fix_cols = ['x_pos','y_pos','pupil_size']
        sacc_cols = ['x_pos','y_pos','end_sacc_x_pos','end_sacc_y_pos','sacc_visual_angle','peak_velocity']
        cols_orderd = ['type','time_stamp','x_pos','y_pos','pupil_size','duration',
               'end_timestamp','end_sacc_x_pos','end_sacc_y_pos','sacc_visual_angle',
               'peak_velocity','input_pin','flags']

        samples, events = self.parse()
        df_samples = pd.DataFrame(samples,columns=['time_stamp','x_pos','y_pos','pupil_size','input_pin','flags'])
        df_samples.replace('.', np.NaN, inplace=True) # missing gaze samples are '.'
        df_samples['type'] = 'gaze'
        
        self._event_to_dataframe(events, 'EFIX', (timestamp_cols + fix_cols), events_dataframes)
        self._event_to_dataframe(events, 'ESACC', (timestamp_cols + sacc_cols), events_dataframes)
        self._event_to_dataframe(events, 'EBLINK', (timestamp_cols), events_dataframes)
        
        df_merged = pd.concat([df_samples,
                       events_dataframes['FIX'],
                       events_dataframes['SACC'],
                       events_dataframes['BLINK']])

        df_eyelink = df_merged[cols_orderd].sort_values('time_stamp')
        return df_eyelink

In [3]:
plr_diksha = raw_eyelink('s04s07_PLR_18Feb22.asc')
plr_diksha.to_dataframe()

,type,time_stamp,x_pos,y_pos,pupil_size,duration,end_timestamp,end_sacc_x_pos,end_sacc_y_pos,sacc_visual_angle,peak_velocity,input_pin,flags
0,gaze,5313018,895.9,519.2,5412.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
1,gaze,5313019,896.8,518.8,5411.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
2,gaze,5313020,897.8,518.5,5411.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
3,gaze,5313021,898.9,518.3,5410.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
4,gaze,5313022,900.0,518.0,5410.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292680,gaze,5631694,918.5,640.6,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
292681,gaze,5631695,918.6,640.9,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
292682,gaze,5631696,918.7,640.6,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
292683,gaze,5631697,918.9,640.5,4599.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...


In [4]:
antisaccade_diksha = raw_eyelink('s04s07_AS_18Feb22.asc')
antisaccade_diksha.to_dataframe()

,type,time_stamp,x_pos,y_pos,pupil_size,duration,end_timestamp,end_sacc_x_pos,end_sacc_y_pos,sacc_visual_angle,peak_velocity,input_pin,flags
0,gaze,2726896,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...
0,sacc,2726897,NaN,NaN,NaN,406,2727302,885.9,615.4,3.2e+006,3270,NaN,NaN
0,blink,2726897,NaN,NaN,NaN,336,2727232,NaN,NaN,NaN,NaN,NaN,NaN
1,gaze,2726897,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...
2,gaze,2726898,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220627,gaze,2978744,1737.1,493.5,2783.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
220628,gaze,2978745,1737.0,493.3,2783.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
220629,gaze,2978746,1736.8,493.2,2783.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
220630,gaze,2978747,1736.7,493.0,2783.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...
